In [33]:
import cv2
import os
import re
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

In [17]:
main_data_folder = 'data/stanford_cars_type'

images = []
labels = []
name = []

for label in os.listdir(main_data_folder):
    label_folder = os.path.join(main_data_folder, label)
    
    for image_file in os.listdir(label_folder):
        image_path = os.path.join(label_folder, image_file)
        
        image = cv2.imread(image_path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)  # Opcjonalne: Zmiana kanałów kolorów

        images.append(image)
        labels.append(label)
        name.append(image_file)
    


label_encoder = LabelEncoder()
labels_encoded = label_encoder.fit_transform(labels)


In [54]:
print(f'{labels_encoded.size}   {len(images)}')
np_images = np.array(images)
print(f'{labels_encoded.T.shape}   {np_images.T.shape}')

concatenated_array = np.vstack((labels_encoded, np_images)).T
print(f'{concatenated_array.shape}')


8144   8144
(8144,)   (3, 255, 255, 8144)


ValueError: all the input arrays must have same number of dimensions, but the array at index 0 has 2 dimension(s) and the array at index 1 has 4 dimension(s)

In [46]:
main_data_folder = 'data/stanford_cars_type'

# Ścieżka do folderu, w którym zostaną zapisane przetworzone zdjęcia
output_folder = 'data/processed'

# Nowy wymiar obrazu
new_size = (255, 255)

# Przetwarzanie i zapis przetworzonych zdjęć
for label in os.listdir(main_data_folder):
    label_folder = os.path.join(main_data_folder, label)
    
    for image_file in os.listdir(label_folder):
        image_path = os.path.join(label_folder, image_file)

        # Wyodrębnij tylko cyfry z początku nazwy
        numeric_prefix = re.match(r'^\d+', image_file).group()

        # Wczytaj obraz za pomocą OpenCV
        image = cv2.imread(image_path)

        # Przeskaluj obraz do nowego wymiaru (255x255) i zamień na skalę szarości
        image = cv2.resize(image, new_size)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

        # Nazwa nowego pliku
        new_filename = f'{numeric_prefix}.jpg'
        new_image_path = os.path.join(output_folder, new_filename)

        # Zapisz przetworzony obraz
        cv2.imwrite(new_image_path, image)

In [55]:
csv_file = 'data/stanford_cars_type.csv'
img_folder = 'data/processed'
data_csv = pd.read_csv(csv_file)
images_tab = []
labels_string = []

for index, row in data_csv.iterrows():
    filename = row['file_name']
    category = row['car_type']

    image_path = os.path.join(img_folder, filename)

    image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    image = image / 255.0
    images_tab.append(image)
    labels_string.append(category)

label_encoder = LabelEncoder()
labels = label_encoder.fit_transform(labels_string)
images = np.array(images_tab)

In [57]:
print(f'{labels.shape}   {images.shape}')


(8144,)   (8144, 255, 255)
